# Mandelbrot Set

### Copyright Information

In [ ]:
# Copyright (c) 2023 Ben Ashbaugh
#
# SPDX-License-Identifier: MIT

## Sample Purpose

This is a port of the [ISPC Mandelbrot](https://github.com/ispc/ispc/tree/master/examples/mandelbrot) sample.
It uses an OpenCL kernel to compute a [Mandelbrot set](https://en.wikipedia.org/wiki/Mandelbrot_set) image, which is displayed in this notebook and then written to a BMP file.

This assuredly is not the fastest Mandelbrot kernel on any OpenCL implementation, but it should perform reasonably well - much better than an equivalent serial implementation!

## Sample

To start the sample, we will import pyopencl and a few other packages that this sample uses.

In [ ]:
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import pyopencl as cl
import argparse
import PIL

We will then define the size of the image we want to generate and we will write our Mandelbrot kernel.
Each OpenCL work-item computes one element of the set, or equivalently, one pixel in the output image.

In [ ]:
width = 768
height = 512

maxIterations = 256

kernelString = """
static inline int mandel(float c_re, float c_im, int count) {
    float z_re = c_re, z_im = c_im;
    int i;
    for (i = 0; i < count; ++i) {
        if (z_re * z_re + z_im * z_im > 4.)
            break;

        float new_re = z_re*z_re - z_im*z_im;
        float new_im = 2.f * z_re * z_im;

        z_re = c_re + new_re;
        z_im = c_im + new_im;
    }

    return i;
}
kernel void Mandelbrot(
    float x0, float y0,
    float x1, float y1,
    int width, int height,
    int maxIterations,
    global int* output)
{
    float dx = (x1 - x0) / width;
    float dy = (y1 - y0) / height;

    float x = x0 + get_global_id(0) * dx;
    float y = y0 + get_global_id(1) * dy;

    int index = get_global_id(1) * width + get_global_id(0);
    output[index] = mandel(x, y, maxIterations);
}
"""

By default, this sample will run on the first platform and device it finds.

To choose a different OpenCL platform, simply change the platform index or device index to a different value.

In [ ]:
if __name__ == "__main__":
    platformIndex = 0
    deviceIndex = 0

    platforms = cl.get_platforms()
    print('Running on platform: ' + platforms[platformIndex].get_info(cl.platform_info.NAME))

    devices = platforms[platformIndex].get_devices()
    print('Running on device: ' + devices[deviceIndex].get_info(cl.device_info.NAME))

As before, we need an OpenCL context to work with and an OpenCL command queue to submit OpenCL commands to the OpenCL device, so create them.

In [ ]:
    context = cl.Context([devices[deviceIndex]])
    commandQueue = cl.CommandQueue(context, devices[deviceIndex])

Once we have an OpenCL context we can create an OpenCL program with the kernel string we created previously, build it, and get our Mandelbrot kernel.

In [ ]:
    program = cl.Program(context, kernelString)
    program.build()
    kernel = program.Mandelbrot

We can also create a buffer to store our Mandelbrot image.

In [ ]:
    deviceMemDst = cl.Buffer(context, cl.mem_flags.ALLOC_HOST_PTR, 
                             width * height * np.uint32().itemsize)

We are now ready to execute our Mandelbrot kernel!

The ND-range for the Mandelbrot kernel will be our image width and height.
The other kernel arguments will be constants used to compute the Mandelbrot set and the buffer where each work-item will write its results.

In [ ]:
    event = kernel(commandQueue, [width, height], None, 
           np.float32(-2.0), np.float32(-1.0), np.float32(1.0), np.float32(1.0),
           np.int32(width), np.int32(height), np.int32(maxIterations), deviceMemDst)

All that's left to do now is to get the results of our Mandelbrot kernel.
We can do this by mapping our output buffer, as usual.
We will do a small amount of post-processing to make the Mandelbrot output more visually appealing.

In [ ]:
    mapped_dst, event = cl.enqueue_map_buffer(commandQueue, deviceMemDst,
                                              cl.map_flags.READ, 
                                              0, width * height, np.uint32)
    with mapped_dst.base:
        colors = np.fromiter((240 if x & 1 else 20 for x in mapped_dst), np.uint8)
        image = Image.fromarray(colors.reshape((height, width)))

Now we can display the results.

In [ ]:
        plt.imshow(image, cmap="gray")

We can also save a bitmap for future offline viewing.

In [ ]:
        filename = 'mandelbrot.bmp'
        image.save(filename)
        print('Wrote image file {}'.format(filename))

This is the end of the Mandelbrot sample.